In [ ]:
import pandas as pd
import geopandas as gpd
import re

import requests


pd.reset_option('display.max_colwidth')
#pd.set_option('display.max_colwidth', None)

### 1. Centroid calculation for all Manhattan Streets - NYC Street Centerline dataset (https://data.cityofnewyork.us/City-Government/NYC-Street-Centerline-CSCL-/exjm-f27b) used
Data dictionary (https://data.cityofnewyork.us/api/views/exjm-f27b/files/cba8af99-6cd5-49fd-9019-b4a6c2d9dff7?download=true&filename=Centerline.pdf)

NB: some of the information in the data dictionary is out of date (last updated in 2016)

In [ ]:
mnhttn_centerlines_res = requests.get('https://data.cityofnewyork.us/resource/8rma-cm9c.geojson?status=2&borocode=1&rw_type=1&$limit=50000')
mnhttn_centerlines_res.status_code

In [ ]:
mnhttn_streets = gpd.GeoDataFrame.from_features(mnhttn_centerlines_res.json()['features']).set_geometry('geometry').set_crs('EPSG:4326')
mnhttn_streets = mnhttn_streets[['r_high_hn', 'r_low_hn', 'l_high_hn', 'l_low_hn', 'full_stree', 'geometry']]
mnhttn_streets

In [ ]:
# get house numbers
mnhttn_streets.insert(loc=4, column='min_hn', value=mnhttn_streets[['r_low_hn', 'l_low_hn']].min(axis=1))
mnhttn_streets.insert(loc=5, column='max_hn', value=mnhttn_streets[['r_high_hn', 'l_high_hn']].max(axis=1))

mnhttn_streets.fillna({
    'min_hn': '',
    'max_hn': ''
}, inplace=True)

mnhttn_streets.insert(loc=6, column='hn_range', value=mnhttn_streets[['min_hn', 'max_hn']].agg(' - '.join, axis=1))
mnhttn_streets.drop(columns=['r_high_hn', 'r_low_hn', 'l_high_hn', 'l_low_hn', 'min_hn', 'max_hn'], inplace=True)

mnhttn_streets['hn_range'] = mnhttn_streets['hn_range'].str.replace(r'^\s-\s$', '', regex=True)

# clean street names
mnhttn_streets['full_stree'] = mnhttn_streets['full_stree'].str.title()

# replacement of abbreviations
mnhttn_streets['full_stree'] = mnhttn_streets['full_stree'].str.replace(r'\bW\b', 'West', regex=True)
mnhttn_streets['full_stree'] = mnhttn_streets['full_stree'].str.replace(r'\bE\b', 'East', regex=True)
mnhttn_streets['full_stree'] = mnhttn_streets['full_stree'].str.replace(r'\bN\b', 'North', regex=True)
mnhttn_streets['full_stree'] = mnhttn_streets['full_stree'].str.replace(r'\bS\b', 'South', regex=True)
mnhttn_streets['full_stree'] = mnhttn_streets['full_stree'].str.replace(r'\bSt\b', 'Street', regex=True)
mnhttn_streets['full_stree'] = mnhttn_streets['full_stree'].str.replace(r'\bAve\b', 'Avenue', regex=True)
mnhttn_streets['full_stree'] = mnhttn_streets['full_stree'].str.replace(r'\bPl\b', 'Place', regex=True)
mnhttn_streets['full_stree'] = mnhttn_streets['full_stree'].str.replace(r'\bLn\b', 'Lane', regex=True)
mnhttn_streets['full_stree'] = mnhttn_streets['full_stree'].str.replace(r'\bAly\b', 'Alley', regex=True)
mnhttn_streets['full_stree'] = mnhttn_streets['full_stree'].str.replace(r'\bDr\b', 'Drive', regex=True)
mnhttn_streets['full_stree'] = mnhttn_streets['full_stree'].str.replace(r'\bSq\b', 'Square', regex=True)
mnhttn_streets['full_stree'] = mnhttn_streets['full_stree'].str.replace(r'\bPlz\b', 'Plaza', regex=True)
mnhttn_streets['full_stree'] = mnhttn_streets['full_stree'].str.replace(r'\bBlvd\b', 'Boulevard', regex=True)
mnhttn_streets['full_stree'] = mnhttn_streets['full_stree'].str.replace(r'\bPkwy\b', 'Parkway', regex=True)
mnhttn_streets['full_stree'] = mnhttn_streets['full_stree'].str.replace(r'\bRd\b', 'Road', regex=True)
mnhttn_streets['full_stree'] = mnhttn_streets['full_stree'].str.replace(r'\bTer\b', 'Terrace', regex=True)
mnhttn_streets['full_stree'] = mnhttn_streets['full_stree'].str.replace(r'\bCir\b', 'Circle', regex=True)
mnhttn_streets['full_stree'] = mnhttn_streets['full_stree'].str.replace(r'\bEn\b', 'Entrance', regex=True)
mnhttn_streets['full_stree'] = mnhttn_streets['full_stree'].str.replace(r'\bEspl\b', 'Esplanade', regex=True)

# transform to ordinal numbers

# function to add st, nd, rd, th to numbers
def to_ordinal(street_name):
    match = re.search(r'\b\d{1,3}\b', street_name)

    if match is not None:
        if (len(match.group(0)) >= 2) and (match.group(0)[-2] == '1'):
            address = street_name[:match.end()] + 'th' + street_name[match.end():]
        elif match.group(0)[-1] == '1':
            address = street_name[:match.end()] + 'st' + street_name[match.end():]
        elif match.group(0)[-1] == '2':
            address = street_name[:match.end()] + 'nd' + street_name[match.end():]
        elif match.group(0)[-1] == '3':
            address = street_name[:match.end()] + 'rd' + street_name[match.end():]
        else:
            address = street_name[:match.end()] + 'th' + street_name[match.end():]
    else:
        return street_name
    
    return address

mnhttn_streets['full_stree'] = mnhttn_streets['full_stree'].apply(lambda row: to_ordinal(row))

# combine and create address
mnhttn_streets.insert(loc=2, column='address', value=mnhttn_streets[['hn_range', 'full_stree']].agg(', '.join, axis=1))
mnhttn_streets['address'] = mnhttn_streets['address'].str.replace(r'^,\s', '', regex=True)
mnhttn_streets['address'] = mnhttn_streets['address'].str.strip()
mnhttn_streets.drop(columns=['hn_range', 'full_stree'], inplace=True)

mnhttn_streets

In [ ]:
# centroid calculation - warning ignored as the results are approximately correct based on visual inspection
mnhttn_streets['centroid'] = mnhttn_streets['geometry'].centroid
mnhttn_streets

### 2. Overlay of taxi zones - NYC Taxi Zones dataset (https://data.cityofnewyork.us/Transportation/NYC-Taxi-Zones/d3c5-ddgc) used

In [ ]:
mnhttn_zones_res = requests.get('https://data.cityofnewyork.us/resource/755u-8jsi.geojson?borough=Manhattan')
mnhttn_zones_res.status_code

In [ ]:
mnhttn_zones = gpd.GeoDataFrame.from_features(mnhttn_zones_res.json()['features']).set_geometry('geometry').set_crs('EPSG:4326')
mnhttn_zones = mnhttn_zones[['location_id', 'zone', 'geometry']]
mnhttn_zones

In [ ]:
# cleaning of zone 103 which appears to be a combination of three zones
mnhttn_zones.loc[19, 'zone'] = 'Liberty Island'
mnhttn_zones.loc[20, 'location_id'] = '104'
mnhttn_zones.loc[20, 'zone'] = 'Ellis Island'
mnhttn_zones.loc[21, 'location_id'] = '105'
mnhttn_zones.loc[21, 'zone'] = "Governor's Island"

### 3. Localisation of centroids in corresponding taxi zones

In [ ]:
# plot all layers
base = mnhttn_streets.plot(figsize = (100,100), color = "grey")
centroid_layer = gpd.GeoSeries(mnhttn_streets["centroid"]).plot(ax = base, color="red", markersize=5)
areas_layer = gpd.GeoSeries(mnhttn_zones["geometry"]).boundary.plot(ax = centroid_layer, color="orange")

In [ ]:
# reset active geometry to centroid column
mnhttn_streets.set_geometry("centroid", inplace=True)

# spatial join to assign the corresponding taxi zone to each point
zoned_streets = gpd.sjoin(mnhttn_streets, mnhttn_zones, how='left', predicate='within')
zoned_streets = zoned_streets.drop('index_right', axis=1)
zoned_streets

In [ ]:
# separate NaNs
zoned_streets_2 = zoned_streets[zoned_streets['location_id'].isna()].drop(columns=['location_id', 'zone'])
zoned_streets = zoned_streets[zoned_streets['location_id'].notna()]

# reset active geometry to centroid column
zoned_streets_2.set_geometry("centroid", inplace=True)

# spatial join to assign the corresponding taxi area to points on the common edges of 2 or more taxi zones
zoned_streets_2 = gpd.sjoin(zoned_streets_2, mnhttn_zones, how='left', predicate='dwithin', distance=0.0005)
zoned_streets_2 = zoned_streets_2.drop('index_right', axis=1)
zoned_streets_2

In [ ]:
# concatenate all dfs
all_streets = pd.concat([zoned_streets, zoned_streets_2], ignore_index=True)
all_streets = all_streets[~all_streets['location_id'].isin(['103', '104', '105', '153', '194', '202'])]
all_streets

In [ ]:
# add zones geometry to df
all_streets = pd.merge(all_streets, mnhttn_zones, on='location_id', how='left').drop('zone_y', axis=1)

all_streets.rename(columns={
    'geometry_x': 'street_geometry',
    'centroid': 'street_centroid',
    'location_id': 'zone_id',
    'zone_x': 'zone_name',
    'geometry_y': 'zone_geometry'
}, inplace=True)

all_streets

In [ ]:
# plot all layers
base = gpd.GeoSeries(all_streets["street_geometry"]).plot(figsize = (50,50), color = "grey")
centroid_layer = gpd.GeoSeries(all_streets["street_centroid"]).plot(ax = base, color="red", markersize=5)
areas_layer = gpd.GeoSeries(all_streets["zone_geometry"]).boundary.plot(ax = centroid_layer, color="orange")

In [ ]:
# transform all zones multipolygons into linestrings
'''
lines = mnhttn_zones.geometry.apply(lambda x: (
    list(map(LineString, zip(x.boundary.coords[:-1], x.boundary.coords[1:])))
    if isinstance(x, Polygon)
    else list(chain(*list(list(map(
        LineString, zip(poly.boundary.coords[:-1], poly.boundary.coords[1:])
    )) for poly in x.geoms)))
)).explode()

# output all edges and zones to which they belong
result = {
    str(line): list(mnhttn_zones.loc[
        (mnhttn_zones.geometry.touches(line)) # line touches the polygon
        & (mnhttn_zones.geometry.intersection(line).length > 0), # And the intersection is more than just a point
        'location_id'
    ].values)
    for line in lines
}

result
'''

### 4. Identification of mobile food vending restrictions - using API behind NYC Mobile Food Vending Restricted Streets Map (https://a816-dohbesp.nyc.gov/IndicatorPublic/mobilefoodvending/)

In [ ]:
# get all nyc restricted streets
nyc_restricted_centerlines = requests.get('https://services3.arcgis.com/A6Zjpzrub8ESZ3c7/arcgis/rest/services/FS_MOVIS_RESTRICTED_STREETS/FeatureServer/0/query?f=geojson&where=RestrictStat%20%3D%20%27Active%27&returnGeometry=true&spatialRel=esriSpatialRelIntersects&maxAllowableOffset=1.0583354500041846&outFields=*&maxRecordCountFactor=4&outSR=102100&resultOffset=0&resultRecordCount=4000&cacheHint=true&quantizationParameters={%22mode%22%3A%22view%22%2C%22originPosition%22%3A%22upperLeft%22%2C%22tolerance%22%3A1.0583354500041846%2C%22extent%22%3A{%22xmin%22%3A978550.3499700576%2C%22ymin%22%3A148097.4157973975%2C%22xmax%22%3A1047926.146913752%2C%22ymax%22%3A239740.22496551275%2C%22spatialReference%22%3A{%22wkid%22%3A102718%2C%22latestWkid%22%3A2263}}}')
nyc_restricted_centerlines.status_code

In [ ]:
# transform to geodf
restricted_streets = gpd.GeoDataFrame.from_features(nyc_restricted_centerlines.json()['features']).set_geometry('geometry').set_crs('EPSG:3857') # when transformed, crs is not so it must be set to original crs
restricted_streets.to_crs('EPSG:4326', inplace=True) # transform to same crs as the all_streets geodf
restricted_streets

In [ ]:
# get Manhattan boundary
mnhttn_boundary_res = requests.get('https://data.cityofnewyork.us/resource/7t3b-ywvw.geojson?BoroName=Manhattan')
mnhttn_boundary_res.status_code

In [ ]:
# transform to geodf
mnhttn_boundary = gpd.GeoDataFrame.from_features(mnhttn_boundary_res.json()['features']).set_geometry('geometry').set_crs('EPSG:4326')
mnhttn_boundary

In [ ]:
# spatial join to filter for Manhattan restrictions
mnhttn_restricted_streets = gpd.sjoin(restricted_streets, mnhttn_boundary, how='left', predicate='within')
mnhttn_restricted_streets = mnhttn_restricted_streets[mnhttn_restricted_streets['boro_code'] == '1']
mnhttn_restricted_streets = mnhttn_restricted_streets.drop('index_right', axis=1)
mnhttn_restricted_streets

In [ ]:
# plot for visual inspection
base = mnhttn_restricted_streets.plot(figsize = (50,50), color = "grey")
areas_layer = gpd.GeoSeries(mnhttn_boundary["geometry"]).boundary.plot(ax = base, color="orange")

In [ ]:
# get unrestricted streets
all_streets.set_geometry('street_centroid', inplace=True)
unrestricted_streets = gpd.sjoin(all_streets, mnhttn_restricted_streets, how='left', predicate='dwithin', distance=0.0002)
computed_restrictions = unrestricted_streets[~unrestricted_streets['Street'].isna()]
unrestricted_streets = unrestricted_streets[unrestricted_streets['Street'].isna()]
unrestricted_streets = unrestricted_streets[['address', 'street_geometry', 'street_centroid', 'zone_id', 'zone_name', 'zone_geometry']]
unrestricted_streets

In [ ]:
all_streets.set_geometry('street_geometry', inplace=True)
# plot only computed restrictions for visual inspection
base = all_streets.plot(figsize = (50,50), color = "grey")
computed_restrictions_layer= gpd.GeoSeries(computed_restrictions["street_geometry"]).plot(ax = base, color="red")

In [ ]:
all_streets.set_geometry('street_geometry', inplace=True)
# plot computed restrictions on top of actual restrictions for visual inspection
base = all_streets.plot(figsize = (50,50), color = "grey")
actual_restrictions_layer = gpd.GeoSeries(mnhttn_restricted_streets["geometry"]).plot(ax = base, color="blue")
computed_restrictions_layer = gpd.GeoSeries(computed_restrictions["street_geometry"]).plot(ax = actual_restrictions_layer, color="red")

Perfect match. The blue lines that are not covered by red ones are not streets, but paths or boardwalks. These are not included in the streets dataset by design, and therefore won't be recommended.

In [ ]:
unrestricted_streets.rename(columns={
    'address': 'street_address'
}, inplace=True)

unrestricted_streets.drop(columns=["zone_name", "zone_geometry"], inplace=True)

unrestricted_streets

### 5. Separate restrictions for storage in their own db table

In [ ]:
mnhttn_restricted_streets.drop(columns=['DOW', 'UNIQID', 'HR', 'RestrictStat', 'Note', 'OBJECTID', 'Shape__Length', 'boro_code', 'boro_name', 'shape_leng', 'shape_area'], inplace=True)
mnhttn_restricted_streets

In [ ]:
# clean DateStart & DateEnd
mnhttn_restricted_streets[['DateStart', 'DateEnd']] = mnhttn_restricted_streets[['DateStart', 'DateEnd']].apply(pd.to_datetime, unit='ms')
mnhttn_restricted_streets['DateStart'] = mnhttn_restricted_streets['DateStart'].dt.strftime('%Y-%m-%d')
mnhttn_restricted_streets['DateEnd'] = mnhttn_restricted_streets['DateEnd'].dt.strftime('%Y-%m-%d')
mnhttn_restricted_streets[['DateStart', 'DateEnd']] = mnhttn_restricted_streets[['DateStart', 'DateEnd']].apply(pd.to_datetime)
mnhttn_restricted_streets

In [ ]:
mnhttn_restricted_streets.rename(columns={
    'geometry': 'restriction_street_geometry',
    'Street': 'restriction_street',
    'FStreet': 'restriction_fstreet',
    'TStreet': 'restriction_tstreet',
    'DateStart': 'restriction_fdate',
    'DateEnd': 'restriction_tdate',
    'MonStr': 'Monday',
    'TueStr': 'Tuesday',
    'WedStr': 'Wednesday',
    'ThuStr': 'Thursday',
    'FriStr': 'Friday',
    'SatStr': 'Saturday',
    'SunStr': 'Sunday'
}, inplace=True)

mnhttn_restricted_streets

In [ ]:
# extract time intervals
mnhttn_restricted_streets['Monday'] = mnhttn_restricted_streets['Monday'].str.extract(r'(\d{1,2}:\d{2}[AP]M\s-\s\d{1,2}:\d{2}[AP]M)')
mnhttn_restricted_streets['Tuesday'] = mnhttn_restricted_streets['Tuesday'].str.extract(r'(\d{1,2}:\d{2}[AP]M\s-\s\d{1,2}:\d{2}[AP]M)')
mnhttn_restricted_streets['Wednesday'] = mnhttn_restricted_streets['Wednesday'].str.extract(r'(\d{1,2}:\d{2}[AP]M\s-\s\d{1,2}:\d{2}[AP]M)')
mnhttn_restricted_streets['Thursday'] = mnhttn_restricted_streets['Thursday'].str.extract(r'(\d{1,2}:\d{2}[AP]M\s-\s\d{1,2}:\d{2}[AP]M)')
mnhttn_restricted_streets['Friday'] = mnhttn_restricted_streets['Friday'].str.extract(r'(\d{1,2}:\d{2}[AP]M\s-\s\d{1,2}:\d{2}[AP]M)')
mnhttn_restricted_streets['Saturday'] = mnhttn_restricted_streets['Saturday'].str.extract(r'(\d{1,2}:\d{2}[AP]M\s-\s\d{1,2}:\d{2}[AP]M)')
mnhttn_restricted_streets['Sunday'] = mnhttn_restricted_streets['Sunday'].str.extract(r'(\d{1,2}:\d{2}[AP]M\s-\s\d{1,2}:\d{2}[AP]M)')
mnhttn_restricted_streets

In [ ]:
# pivot to long
mnhttn_restricted_streets = pd.melt(mnhttn_restricted_streets, id_vars=['restriction_street_geometry', 'restriction_street', 'restriction_fstreet', 'restriction_tstreet', 'restriction_fdate', 'restriction_tdate'], value_vars=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], var_name='restriction_weekday', value_name='time_range')
mnhttn_restricted_streets

In [ ]:
# keep valid restrictions (e.g. a restriction can be active on weekdays not weekends -> remove weekend days)
mnhttn_restricted_streets = mnhttn_restricted_streets[~mnhttn_restricted_streets['time_range'].isna()]
mnhttn_restricted_streets

In [ ]:
# split time interval
mnhttn_restricted_streets[['restriction_ftime', 'restriction_ttime']] = mnhttn_restricted_streets['time_range'].str.split(' - ', n=1, expand=True)
mnhttn_restricted_streets

In [ ]:
# further cleaning
mnhttn_restricted_streets.drop(columns=['time_range'], inplace=True)

mnhttn_restricted_streets['restriction_ftime'] = pd.to_datetime(mnhttn_restricted_streets['restriction_ftime'], format='%I:%M%p').dt.strftime('%H:%M')
mnhttn_restricted_streets['restriction_ttime'] = pd.to_datetime(mnhttn_restricted_streets['restriction_ttime'], format='%I:%M%p').dt.strftime('%H:%M')

mnhttn_restricted_streets

In [ ]:
# reorder, reset index, drop unnecessary col, insert col for id 

ordered_cols = ['restriction_street', 'restriction_fstreet', 'restriction_tstreet', 'restriction_fdate', 'restriction_tdate', 'restriction_street_geometry', 'restriction_weekday', 'restriction_ftime', 'restriction_ttime']
mnhttn_restricted_streets = mnhttn_restricted_streets[ordered_cols]

mnhttn_restricted_streets.reset_index(inplace=True)

mnhttn_restricted_streets.drop(columns=['index'], inplace=True)

mnhttn_restricted_streets.insert(loc=0, column='restriction_id', value=mnhttn_restricted_streets.index)

mnhttn_restricted_streets